In [15]:
import multiprocessing
from multiprocessing.pool import ThreadPool
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import StarmapParallelization
from pymoo.constraints.as_obj import ConstraintsAsObjective
from pymoo.constraints.as_penalty import ConstraintsAsPenalty
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize

import optimize as op

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


In [16]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.4
opt.getLiftoffState()
print("Initial Cost", opt.objective(opt.bp_init_guess))


class CorgiProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        super().__init__(n_var=24,
                         n_obj=1,
                         n_ieq_constr=24,
                         xl=np.array([0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05,
                                      0.03, 0.00, 0.01, -0.05, 0.01, -0.05]),
                         xu=np.array([0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5,
                                      0.04, 0.02, 1, 0.5, 1, 0.5]),
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        f  = opt.objective(x)

        g1 = -1 * opt.innerWsPFConstraint(x)
        g2 = -1 * opt.outerWsPFConstraint(x)
        g3 = np.abs(opt.velocityConstraint(x)) - 0.01
        print("g3", np.average(g3))
        g = np.hstack((g1, g2))
        g = np.hstack((g, g3))

        out["F"] = [f]
        out["G"] = list(g)

# initialize the thread pool and create the runner
# n_proccess = 8
# pool = multiprocessing.Pool(n_proccess)
# runner = StarmapParallelization(pool.starmap)

# initialize the thread pool and create the runner
n_threads = 16
pool = ThreadPool(n_threads)
runner = StarmapParallelization(pool.starmap)
problem = CorgiProblem(elementwise_runner=runner, elementwise_evaluation=True)
# problem = CorgiProblem()


[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
[0.05 0.01 0.1  0.01 0.1  0.01]
iter 1 cost -1.129639564392515
Initial Cost -1.129639564392515


In [17]:
algorithm = NSGA2(
    pop_size=60,
    n_offsprings=30,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)
# algorithm = PSO()

In [18]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 100)

In [19]:

result = minimize(ConstraintsAsPenalty(problem, 1.0),
                  algorithm,
                  termination,
                  seed=1,
                  save_history=False,
                  verbose=False)

pool.close()


[0.03417022 0.01440649 0.01011323 0.11628291 0.15528833 0.00078623]
[0.0318626  0.00691121 0.4027998  0.2463492  0.42500257 0.32687073]
[0.03204452 0.01756235 0.03711372 0.31875713 0.42313175 0.25727941]
[0.03140387 0.00396203 0.80273712 0.48254387 0.32028994 0.33077744]
[ 0.03876389  0.01789213  0.09419377 -0.02851987  0.17813212  0.43297838]
[0.03098347 0.00842215 0.95831063 0.24324091 0.69495834 0.1235336 ]
[0.03686501 0.01669251 0.02810539 0.36257937 0.98897248 0.36149111]
[0.03280444 0.01578559 0.11219375 0.19634144 0.90950955 0.11148778]
[0.03287775 0.00260057 0.02917329 0.32335954 0.21951183 0.09605066]
[0.03491573 0.00106725 0.57837643 0.03070072 0.59341248 0.3348671 ]
[0.03102334 0.00828112 0.69745616 0.1777986  0.05945392 0.24474302]
[0.03663795 0.01029778 0.94514881 0.27260527 0.9043679  0.02561109]
[0.03139276 0.01614783 0.40370007 0.04094481 0.92823349 0.14127122]
[0.03750812 0.01451996 0.88447303 0.29301971 0.75343301 0.14189409]
[0.03269928 0.01791772 0.43381028 0.480662

In [20]:
print(result.X)
print(result.F)
min_ = np.min(result.F, axis=0)
opt.exportBezierData(result.X)

[ 3.02241469e-02  6.18009603e-04  1.37761695e-02 -3.18624200e-02
  1.73132673e-02 -4.83157169e-02  3.00001278e-02  4.80521588e-05
  1.05416270e-02 -2.11752489e-02  1.22629188e-02 -2.58490072e-02
  3.02090413e-02  6.08501068e-03  1.03035476e-02 -1.39436626e-02
  1.44150362e-02 -3.65158419e-02  3.01523141e-02  6.22211231e-04
  1.22612729e-02 -2.63632220e-02  1.71104904e-02 -4.33872217e-02]
[-0.15465748]
sbrio exported to ./csv_trajectory/20230828/2023-08-29_traj_400_16.csv
webot exported /home/guanlunlu/corgi_webots/controllers/supervisor/2023-08-29_traj_400_16.csv


In [21]:
opt = op.corgiOptimize()
opt.L = 0.2
opt.T_sw = 0.6
opt.getLiftoffState()
print("Initial Cost", opt.objective(result.X))

[ 0.03022415  0.00061801  0.01377617 -0.03186242  0.01731327 -0.04831572]
[ 3.00001278e-02  4.80521588e-05  1.05416270e-02 -2.11752489e-02
  1.22629188e-02 -2.58490072e-02]
[ 0.03020904  0.00608501  0.01030355 -0.01394366  0.01441504 -0.03651584]
[ 0.03015231  0.00062221  0.01226127 -0.02636322  0.01711049 -0.04338722]
iter 1 cost -2.287768243034658
Initial Cost -2.287768243034658
